In [1]:
# Importing the Keras libraries and packages

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (48, 48, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 7, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


Using TensorFlow backend.


In [2]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/Training',
                                                 target_size = (48, 48),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('dataset/PrivateTest',
                                            target_size = (48, 48),
                                            batch_size = 32,
                                            class_mode = 'categorical')
   
classifier.fit_generator( training_set, 
                         steps_per_epoch = 28707,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 3589 
                        )

Found 28707 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/5
28707/28707 [==============================] - 2835s 99ms/step - loss: 1.1264 - acc: 0.5729 - val_loss: 1.2438 - val_acc: 0.5587
Epoch 2/5
28707/28707 [==============================] - 2518s 88ms/step - loss: 0.8271 - acc: 0.6940 - val_loss: 1.4211 - val_acc: 0.5653
Epoch 3/5
28707/28707 [==============================] - 2324s 81ms/step - loss: 0.7030 - acc: 0.7415 - val_loss: 1.5047 - val_acc: 0.5704
Epoch 4/5
28707/28707 [==============================] - 2459s 86ms/step - loss: 0.6407 - acc: 0.7654 - val_loss: 1.6123 - val_acc: 0.5726
Epoch 5/5
28707/28707 [==============================] - 2440s 85ms/step - loss: 0.5929 - acc: 0.7835 - val_loss: 1.6938 - val_acc: 0.5628


In [ ]:
#save model
classifier.save('face_expression_model.h5')

In [1]:
from keras.models import load_model
model=load_model('face_expression_model.h5')

Using TensorFlow backend.


In [ ]:
# Run time emotion detection
import cv2
size = 4
# We load the xml file
classifier1 = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

capture = cv2.VideoCapture(0) #Using default WebCam connected to the PC.

while capture.isOpened():
    (rval, im) = capture.read()
    im=cv2.flip(im,1,0) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (int(im.shape[1]/size), int(im.shape[0]/size)))
	
    # detect MultiScale / faces 
    faces = classifier1.detectMultiScale(mini)
	
    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        cv2.rectangle(im, (x,y), (x+w,y+h), (0,255,0), 4)
        
        #Save just the rectangle faces in SubRecFaces
        sub_face = im[y:y+h, x:x+w]
        
        cv2.imwrite("predict_image.jpg",sub_face)
        capture.release()
        cv2.waitKey(0)
    
    if cv2.waitKey(1) & 0xFF == ord("q")  :
        break

test_image = cv2.imread("/home/keshav/Music/all/predict_image.jpg")

test_image = cv2.resize(test_image,(48,48))
cv2.imwrite("final.jpg",test_image)
test_image = test_image.reshape((-1,48,48,3))

result = model.predict(test_image)
if result[0][0] == 1:
    ans = 'ANGRY'
elif result[0][1] == 1:
    ans = 'DISGUST'
elif result[0][2] == 1:
    ans = 'FEAR'
elif result[0][3] == 1:
    ans = 'HAPPY'
elif result[0][4] == 1:
    ans = 'SAD'
elif result[0][5] == 1:
    ans = 'SURPRISE'
else :
    ans = 'NEUTRAL'
print(ans)